In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By

cols = ['Nama', 'Harga Diskon', 'Harga Awal', 'Diskon', 'Cashback', 'Terjual', 'Kota/Kab', 'Toko', 'Rating']
df = pd.DataFrame(columns = cols)

service = Service("C:\Program Files\edgedriver_win64")
driver = webdriver.Edge(service = service)

url = 'https://www.tokopedia.com/search?st=&q=lighting&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource='

driver.get(url)
driver.implicitly_wait(5)

In [2]:
def is_float(string):
    try:
        float(string)
        return True
    
    except ValueError:
        return False

In [15]:
res = driver.find_elements(By.XPATH, "//div[@class='css-jza1fo']")

for k in range(len(res)):
    list_items = res[k].text.split('\n')

    i = 0
    while i < len(list_items):
        nama = ''
        harga_diskon = ''
        harga_awal = ''
        diskon = ''
        cashback = ''
        terjual = ''
        kota_kab = ''
        toko = ''
        rating = ''

        count = i
        j = 0
        ada_harga = False
        while (j < 11) and (count + j < len(list_items)):

            if len(list_items[count + j]) > 20:
                if nama == '':
                    nama = list_items[count + j]

                elif toko != '':
                    j = 11

            elif list_items[count + j].startswith('Rp'):
                if ada_harga:
                    harga_awal = list_items[count + j]

                else:
                    harga_diskon = list_items[count + j]
                    ada_harga = True

            elif (not list_items[count + j].startswith('Cashback')) and (list_items[count + j].endswith('%')):
                diskon = list_items[count + j]

            elif list_items[count + j].startswith('Cashback'):
                cashback = list_items[count + j].split(' ')[1]

            elif is_float(list_items[count + j]):
                kota_kab = list_items[count + j - 2]
                toko = list_items[count + j - 1]
                rating = list_items[count + j]

            elif list_items[count + j].endswith('terjual'):
                terjual = list_items[count + j].split(' ')[0]

            j += 1

        if harga_awal == '':
            harga_awal = harga_diskon
            harga_diskon = ''

        temp = [nama, harga_diskon, harga_awal, diskon, cashback, terjual, kota_kab, toko, rating]

        if i != len(list_items) - 1:
            while list_items[i] != nama:
                i += 1

        i += len([_ for _ in temp if _ != ''])

        df.loc[len(df)] = temp

In [16]:
df

,Nama,Harga Diskon,Harga Awal,Diskon,Cashback,Terjual,Kota/Kab,Toko,Rating
0,INBEX 150W Lampu lighting studio standing lamp...,Rp266.000,Rp660.000,60%,17%,250+,Kab. Bekasi,INBEX Official Store,4.8
1,Ulanzi VL120 RGB LED Video 20 Light Effect Lam...,Rp285.000,Rp395.000,28%,10%,250+,Jakarta Utara,Cubus_Co_ID,5.0
2,"Lifesmart Cololight Music Light Bar, Lampu LED...",,Rp370.000,,2%,1,Jakarta Utara,Kotakbit,5.0
3,EZVIZ LB1-Color Smart Light Bulb 8W Wifi,Rp39.000,Rp95.000,59%,4%,1rb+,Jakarta Utara,CCTVDahua,4.9
4,Slim Light Box LED Frame Advertising Portable ...,Rp900.000,Rp1.250.000,28%,6%,100+,Jakarta Selatan,Visual Light,5.0
...,...,...,...,...,...,...,...,...,...
271,LED MAGNETIC TRACK LIGHT SYSTEM GRILLE LIGHT,Rp90.000,Rp100.000,10%,8%,100+,Jakarta Utara,The Dens Living,5.0
272,lampu laser sorot proyektor tidur cahaya binta...,,Rp155.000,,3%,500+,Jakarta Utara,Three Acc,4.9
273,Sensor Lampu Otomatis Automatic Day Light Swit...,Rp22.147,Rp27.342,19%,,1rb+,Jakarta Barat,Chint Home Electrics,4.9
274,Solar light Lampu Tenaga Surya Jalan Luar Ruan...,Rp133.999,Rp153.000,12%,10%,250+,Tangerang,ookas,4.6
